In [16]:
import pandas as pd
import numpy as np
import pickle as pk
import re
import warnings

from gensim.models import FastText
from sentence_transformers import SentenceTransformer
from tqdm.notebook import tqdm
from nltk import sent_tokenize
from sklearn.metrics.pairwise import cosine_similarity

In [11]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
import os
PATH = '/Users/yoonjeong/Desktop/Lab_project/'
os.chdir(PATH)

In [2]:
df = pd.read_csv("2.Preprocessing/fraud_prep.csv", encoding="cp949")

In [5]:
# 9만자 미만의 글만 남김
temp = df[df['판례내용_전처리'].str.len() < 90000]

In [6]:
model = SentenceTransformer('jhgan/ko-sbert-sts')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.44k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/538 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [27]:
temp.판례내용_전처리[0]

'이    유  상고이유를 판단한다.  . 불가벌적 사후행위에 대한 판단유사수신행위의 규제에 관한 법률이하 유사수신행위법이라 한다 제조 제항 제조를 위반한 행위는 그 자체가 사기행위에 해당한다거나 사기행위를 반드시 포함한다고 할 수 없고 유사수신행위법 위반죄가 형법 제조 제항의 사기죄와 구성요건을 달리하는 별개의 범죄로서 서로 보호법익이 다른 이상 유사수신행위를 한 자가 출자자에게 별도의 기망행위를 하여 유사수신행위로 조달받은 자금의 전부 또는 일부를 다시 투자받는 행위는 유사수신행위법 위반죄와 다른 새로운 보호법익을 침해하는 것으로서 유사수신행위법 위반죄의 불가벌적 사후행위가 되는 것이 아니라 별죄인 사기죄를 구성한다.  원심은 판시와 같은 이유로 피고인의 . . .경 피해자 공소외인에 대한 판시 사기의 점에 관한 공소사실에 대하여 유사수신행위법 제조에서 금지하는 유사수신행위에는 기망행위가 포함되어 있지 않고 유사수신행위법 위반죄와 사기죄는 구성요건을 달리하는 별개의 범죄로서 서로 행위의 태양이나 보호법익을 달리한다는 등의 이유로 기존 범죄인 유사수신행위법 위반죄의 가벌적 평가범위 내에 흡수되는 불가벌적 사후행위에 해당하지 않는다고 판단하였다.  원심판결 이유를 위 법리 및 기록에 비추어 살펴보면 이 부분 원심판단에 일부 적절하지 않은 점이 있으나 피고인의 . . .경 피해자 공소외인에 대한 판시 사기죄가 유사수신행위법 위반죄의 불가벌적 사후행위에 해당하지 않는다고 본 원심의 판단에 불가벌적 사후행위에 관한 법리를 오해함으로써 판결에 영향을 미친 잘못이 없다.  . 나머지 상고이유에 대한 판단  이 부분 상고이유는 형사소송법 제조 제호에서 정한 형보다 가벼운 형이 선고된 이 사건에서 사실오인법리오해를 내세우며 실질적으로 원심의 증거 선택 및 증명력에 관한 판단 또는 이에 기초한 사실인정을 탓하는 것이거나 원심이 인정한 사실과 다른 사실관계를 전제로 법리오해를 지적하는 취지의 주장 또는 형이 너무 무거워 부당하다는 취지의 주장에 해당하여 모두 적법한 상고이유로

In [12]:
sentence_tokened = []

for x in tqdm(temp['판례내용_전처리']):
    sentence_tokened.append(sent_tokenize(x))

  0%|          | 0/2250 [00:00<?, ?it/s]

In [13]:
sentence_tokened

Output hidden; open in https://colab.research.google.com to view.

In [14]:
embeddings = []

for sentences in tqdm(sentence_tokened):
    embedding = None
    count = 0
    for sentence in sentences:
        if embedding is None:
            embedding = model.encode(sentence)
        else:
            embedding += model.encode(sentence)
        count += 1
    embeddings.append(embedding / count)

  0%|          | 0/2250 [00:00<?, ?it/s]

In [15]:
with open("3.Modeling/sbert_embedding.pickle", "wb") as fw:
    pk.dump(embeddings, fw)